# ObsidianTools examples

In [1]:
import sys 
sys.path.append('..')

import os
from dotenv import load_dotenv
load_dotenv('.env', override=True)

import obsidiantools.api as otools
from obsidiantools.md_utils import get_tags
from pprint import pprint
import markdown
import frontmatter

In [2]:
VAULT_DIR = os.environ.get('OBSIDIAN_VAULT_DIR')
VAULT_DIR

'/Users/dvolk/repo/cobrain-dev/vault'

In [3]:
class Section:
    def __init__(self, title, level):
        self.title = title
        self.level = level
        self.content = []
        self.subsections = []

    def add_content(self, content):
        self.content.append(content)

    def add_subsection(self, subsection):
        self.subsections.append(subsection)

    def __repr__(self):
        return f"Section(title={self.title}, level={self.level}, content={self.content}, subsections={self.subsections})"


In [4]:
def parse_markdown(file_path):
    root = Section(title="root", level=0)
    current_section = root

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('#'):
                # Count the level of the section
                level = len(line.split(' ')[0])
                title = line[level+1:].strip()

                # Create a new section
                new_section = Section(title=title, level=level)

                # Find the correct parent for the new section
                while current_section.level >= level:
                    current_section = current_section.parent
                current_section.add_subsection(new_section)
                
                # Update current section
                new_section.parent = current_section
                current_section = new_section
            else:
                # Add line to the content of the current section
                current_section.add_content(line)

    return root


In [5]:
MAX_FILES = 10

def load_markdown_files(directory):
    file_count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            print(filename)
            filepath = os.path.join(directory, filename)
            # tags = get_tags(filepath, show_nested=True)
            # print(f"Tags: {tags}") if tags else None

            with open(filepath) as f:
                metadata, content = frontmatter.parse(f.read())
            
            print(f"Frontmatter: {metadata}")

            sections = parse_markdown(filepath)

            yield filepath, metadata.get("tags", []), sections

            file_count += 1
            if file_count >= MAX_FILES:
                break

            # with open(filepath, 'r', encoding='utf-8') as file:
            #     text = file.read()
            #     content = markdown.markdown(text)
            #     metadata = content.metadata if hasattr(content, 'metadata') else {}
            #     tags = metadata.get('tags', '').split(', ') if 'tags' in metadata else []
            #     yield content, tags

In [6]:
for content, tags, sections in load_markdown_files(VAULT_DIR + '/notes'):
    print(content)
    print(tags)
    print(sections)
    print()
    print('---')

Lessons on ML Platforms — from Netflix, DoorDash, Spotify, and more _ by Ernest Chan _ May, 2021 _ Towards Data Science.md


AttributeError: module 'frontmatter' has no attribute 'parse'

In [ ]:
vault = otools.Vault("/Users/dvolk/repo/cobrain/vault/").connect(attachments=False).gather()

In [ ]:
vault.get_all_file_metadata()

In [ ]:
vault.md_file_index